In [2]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [21]:
class XgbClassifier(object):
    def __init__(self):
        self.clf = None

    def get_clf(self):
        return self.clf
    
    def get_xgb_param(self):
        param = {}
        param['booster'] = 'gbtree'
        param['objective'] = 'multi:softprob'
        param['num_class'] = 3
        param['eval_metric'] = 'mlogloss'
        param['eta'] = 0.05
        param['n_estimators'] = 20
        param['max_depth'] = 6
        param['min_child_weight'] = 0.8
        param['colsample_bytree'] = 0.5
        param['subsample'] = 0.5
        param['silent'] = 1
        param['nthread'] = 4
        param['seed'] = 777

        return param

    def fit(self, X, y,
            test_size=0.3,
            n_fold=10,
            get_best_round=True,
            round_num=1000):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

        if get_best_round:
            d_all = xgb.DMatrix(data=X, label=y)

            print('XGB cv to get best round...')
            result = xgb.cv(self.get_xgb_param(),
                            d_all,
                            nfold=n_fold,
                            num_boost_round=2000,
                            early_stopping_rounds=100)

            best_round = len(result)
            print('best round = {}'.format(best_round))

            round_num = best_round

        d_train = xgb.DMatrix(data=X_train, label=y_train)
        d_test = xgb.DMatrix(data=X_test, label=y_test)

        print('XGB Training...')
        clf = xgb.train(self.get_xgb_param(),
                        d_train,
                        round_num,
                        evals=[(d_test, 'test')],
                        early_stopping_rounds=50)

        
        
        self.clf = clf
        
        return self.clf
    
    def predict(self, X):
        data = xgb.DMatrix(data=X)
        y_pred = self.clf.predict(data=data)
        y_pred_cat = np.argmax(y_pred, axis=1)

        return y_pred_cat    
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return accuracy_score(y_test, y_pred)

    def precision(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return precision_score(y_test, y_pred, average=None)

    def recall(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return recall_score(y_test, y_pred, average=None)    
    

In [23]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

xgb_classifier = XgbClassifier()
_ = xgb_classifier.fit(X, y)

print('accuracy = {}'.format(xgb_classifier.score(X, y)))
print('precision = {}'.format(xgb_classifier.precision(X, y)))
print('recall = {}'.format(xgb_classifier.recall(X, y)))

XGB cv to get best round...
best round = 114
XGB Training...
[0]	test-mlogloss:1.06297
Will train until test-mlogloss hasn't improved in 50 rounds.
[1]	test-mlogloss:1.03103
[2]	test-mlogloss:0.979451
[3]	test-mlogloss:0.932437
[4]	test-mlogloss:0.892446
[5]	test-mlogloss:0.84826
[6]	test-mlogloss:0.817596
[7]	test-mlogloss:0.785726
[8]	test-mlogloss:0.756402
[9]	test-mlogloss:0.72402
[10]	test-mlogloss:0.69631
[11]	test-mlogloss:0.67841
[12]	test-mlogloss:0.651386
[13]	test-mlogloss:0.625226
[14]	test-mlogloss:0.601737
[15]	test-mlogloss:0.578046
[16]	test-mlogloss:0.558059
[17]	test-mlogloss:0.542582
[18]	test-mlogloss:0.524389
[19]	test-mlogloss:0.505394
[20]	test-mlogloss:0.49301
[21]	test-mlogloss:0.482382
[22]	test-mlogloss:0.466287
[23]	test-mlogloss:0.453059
[24]	test-mlogloss:0.439728
[25]	test-mlogloss:0.432332
[26]	test-mlogloss:0.421146
[27]	test-mlogloss:0.408752
[28]	test-mlogloss:0.398067
[29]	test-mlogloss:0.388167
[30]	test-mlogloss:0.378358
[31]	test-mlogloss:0.370947

In [10]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])